# Testnotebook und Embeddings

Nachdem die python api jetzt angeglichen und vereinfacht wurden ist, wollte ich mal in nem Notebook zeigen wie die so benutzt werden kann. Gleichzeitig hab ich den query manager noch erweitert mit der Fähigkeit, neue embeddings zu berechnen und in der knowledgebase zu speichern. 

Zuerst als Baseline mal mit einer simplen Frage die verschiedenen embedding modelle vergleichen. 
Das 'Standard'-Embedding ist 'BAAI/bge-large-en-v1.5', was nur auf englischen Daten trainiert wurde. Ich habe noch zusätzlich die 'intfloat/multilingual-e5-large' embeddings berechnet und in die knowledge base eingefügt.

In [2]:
simple_question_english = "How many patients under 18 yo had a CONDITION while on DRUG"
simple_question_german = "Wie viele Patient:innen unter 18 Jahren hatten eine CONDITION während sie DRUG einnahmen?"

In [3]:
import rag_helper as rh
import pandas as pd
lib_path = "../querylib_20250825.db"

results_simple_mono = rh.match(
    file=lib_path, 
    query=[simple_question_english, simple_question_german], # schicke methode um mehrere queries zu matchen hintereinander
    k=3,
    embedding_model="BAAI/bge-large-en-v1.5"
)[["SourceQuery","QUESTION_MASKED","INDEX", "QUESTION_TYPE", "Score"]] # das filtered das resultierende df gleich mal


In [4]:
with pd.option_context("display.max_rows", None,
                       "display.max_colwidth", None):
    display(results_simple_mono)

,SourceQuery,QUESTION_MASKED,INDEX,QUESTION_TYPE,Score
0,How many patients under 18 yo had a CONDITION while on DRUG,How many patients > 18 yo have CONDITION and are taking DRUG?,46,QA,0.940201
1,How many patients under 18 yo had a CONDITION while on DRUG,How many patients in age between 18 and 35 or in age between 40 and 75 have CONDITION and have taken DRUG at least once?,49,QA,0.871003
2,How many patients under 18 yo had a CONDITION while on DRUG,What is the average age of patients with CONDITION taking DRUG?,44,QA,0.867502
3,Wie viele Patient:innen unter 18 Jahren hatten eine CONDITION während sie DRUG einnahmen?,How many patients > 18 yo have CONDITION and are taking DRUG?,46,QA,0.741935
4,Wie viele Patient:innen unter 18 Jahren hatten eine CONDITION während sie DRUG einnahmen?,What is the average age of patients with CONDITION taking DRUG?,44,QA,0.737182
5,Wie viele Patient:innen unter 18 Jahren hatten eine CONDITION während sie DRUG einnahmen?,How many patients in age between 18 and 35 or in age between 40 and 75 have CONDITION and have taken DRUG at least once?,49,QA,0.732680


Hier wird das embedding von "SourceQuery" verglichen mit den embeddings von "QUESTION_MASKED". Lustigerweise ergibt die deutsche version den gleichen top hit, aber mit dem bedeutend niedrigen score. Auch die Abstände zwischen den schlechteren Hits sind viel kleiner als im englischen. Auf das englische embedding sollte man sich für deutsche queries also nicht verlassen, aber das wird niemanden überraschen. 

Wie performed das multilinguale embedding?

In [5]:
results_simple_multi = rh.match(
    file=lib_path, 
    query=[simple_question_english, simple_question_german],
    k=3,
    embedding_model="intfloat/multilingual-e5-large"
)[["SourceQuery","QUESTION_MASKED","INDEX", "QUESTION_TYPE", "Score"]]


In [6]:
with pd.option_context("display.max_rows", None,
                       "display.max_colwidth", None):
    display(results_simple_multi)

,SourceQuery,QUESTION_MASKED,INDEX,QUESTION_TYPE,Score
0,How many patients under 18 yo had a CONDITION while on DRUG,How many patients > 18 yo have CONDITION and are taking DRUG?,46,QA,0.954108
1,How many patients under 18 yo had a CONDITION while on DRUG,How many patients in age between 18 and 35 have CONDITION?,5,QA,0.924168
2,How many patients under 18 yo had a CONDITION while on DRUG,How many patients in age between 18 and 35 or in age between 40 and 75 have CONDITION and have taken DRUG at least once?,49,QA,0.919206
3,Wie viele Patient:innen unter 18 Jahren hatten eine CONDITION während sie DRUG einnahmen?,How many patients > 18 yo have CONDITION and are taking DRUG?,46,QA,0.931067
4,Wie viele Patient:innen unter 18 Jahren hatten eine CONDITION während sie DRUG einnahmen?,How many patients in age between 18 and 35 or in age between 40 and 75 have CONDITION and have taken DRUG at least once?,49,QA,0.903394
5,Wie viele Patient:innen unter 18 Jahren hatten eine CONDITION während sie DRUG einnahmen?,What is the average age of patients with CONDITION taking DRUG?,44,QA,0.899197


Auch wieder die gleichen top hits, diesmal sind aber die nachfolge hits auch anders. Dafür ist der Abstand zum zweiten bei beiden gleich. Ich denke, das ist ein guter Indikator?

## Kohorten-Kritierien
Vielleicht ist eine deutlich komplexeres matching aufschlussreicher? Leider sind komplexere Strukturen schwerer aufzudrüseln für reine embedding-basierte Suche, deswegen machen wir diesen ganzen Umweg mit mehreren LLM-based steps dazwischen. Das RAG system soll einfach kontext liefern, was ungefähr im gleichen ballpark spielt. Deswegen wird sich die Sprach-Frage eher auf die LLM system prompts konzentrieren vermute ich.

Um einfach mal zu sehen, ob das Multilinguale embedding wenigstens einen genauen Match 'wiederfinden' kann, habe ich eine Cohort-Question straight aus der Datenbank genommen und per LLM auf deutsch übersetzen lassen. 

In [7]:
cohort_question_english = """Index date description: First prescription of DRUG_CLASS

Inclusion criteria: Age 12-17 years at index date
First prescription between 2017-2021
At least one MEASUREMENT within 30 days before index
At least one CONDITION diagnosis before index
Continuous observation for 180 days before index

Exclusion criteria: Prior DRUG_CLASS use in 180 days before index
CONDITION diagnosis before index
CONDITION during study period
CONDITION diagnosis in year before index
"""

cohort_question_german = """Indexdatumsbeschreibung: Erste Verschreibung einer DRUG_CLASS

Einschlusskriterien: Alter 12–17 Jahre zum Indexdatum
Erste Verschreibung zwischen 2017–2021
Mindestens eine MEASUREMENT innerhalb von 30 Tagen vor dem Indexdatum
Mindestens eine CONDITION-Diagnose vor dem Indexdatum
Durchgehende Beobachtungszeit von 180 Tagen vor dem Indexdatum

Ausschlusskriterien: Vorherige Anwendung von DRUG_CLASS in den 180 Tagen vor dem Indexdatum
CONDITION-Diagnose vor dem Indexdatum
CONDITION während des Studienzeitraums
CONDITION-Diagnose im Jahr vor dem Indexdatum
"""

In [8]:
results_cohort_mono = rh.match(
    file=lib_path, 
    query=[cohort_question_english, cohort_question_german],
    k=3,
    embedding_model="BAAI/bge-large-en-v1.5"
)[["SourceQuery","QUESTION_MASKED","INDEX", "QUESTION_TYPE", "Score"]]

results_cohort_multi = rh.match(
    file=lib_path, 
    query=[cohort_question_english, cohort_question_german], 
    k=3,
    embedding_model="intfloat/multilingual-e5-large"
)[["SourceQuery","QUESTION_MASKED","INDEX", "QUESTION_TYPE", "Score"]]

### Monolinguales Embedding:

In [9]:
display(results_cohort_mono)

,SourceQuery,QUESTION_MASKED,INDEX,QUESTION_TYPE,Score
0,Index date description: First prescription of ...,Index date description: First prescription of ...,139,COHORT_GENERATOR,1.000000
1,Index date description: First prescription of ...,Index date description: First prescription of ...,151,COHORT_GENERATOR,0.972796
2,Index date description: First prescription of ...,Index date description: First DRUG_CLASS presc...,177,COHORT_GENERATOR,0.970792
3,Indexdatumsbeschreibung: Erste Verschreibung e...,Index date description: First prescription of ...,164,COHORT_GENERATOR,0.804558
4,Indexdatumsbeschreibung: Erste Verschreibung e...,Index date description: First DRUG_CLASS presc...,159,COHORT_GENERATOR,0.804509
5,Indexdatumsbeschreibung: Erste Verschreibung e...,Index date description: First prescription of ...,139,COHORT_GENERATOR,0.803903


Das originale embedding sieht natürlich gleich, dass es ein identisches Match gibt mit der englischen query. Aber wenn man ihm die übersetzte query gibt, ist ende gelände.

### Multilinguales Embedding:

In [10]:
display(results_cohort_multi)

,SourceQuery,QUESTION_MASKED,INDEX,QUESTION_TYPE,Score
0,Index date description: First prescription of ...,Index date description: First prescription of ...,139,COHORT_GENERATOR,1.000000
1,Index date description: First prescription of ...,Index date description: First DRUG_CLASS presc...,177,COHORT_GENERATOR,0.983743
2,Index date description: First prescription of ...,Index date description: First prescription fil...,150,COHORT_GENERATOR,0.982857
3,Indexdatumsbeschreibung: Erste Verschreibung e...,Index date description: First prescription of ...,139,COHORT_GENERATOR,0.948063
4,Indexdatumsbeschreibung: Erste Verschreibung e...,Index date description: First DRUG_CLASS presc...,177,COHORT_GENERATOR,0.937752
5,Indexdatumsbeschreibung: Erste Verschreibung e...,Index date description: First prescription fil...,150,COHORT_GENERATOR,0.934161


Wenn man das multilinguale embedding verwendet, ist der top match in der Tat die Original-Query. Mit deutlich niedrigerem score aber immerhin.

## resumee
gibt nicht so wirklich eins. die letzte zeit war primär ausreifung von der RAG infrastruktur, jetzt kann mal sowohl pythonic als auch per terminal die funktionen nutzen. Die relevanteste Sache ist natürlich das Matching, aber vielleicht werden die anderen Sachen auch mal relevant. Man kann mit verschiedenen Embeddings rumspielen, aber ich vermute das LLM-System prompts fast eine bedeutentere Rolle spielen.

Ich sehe schon eine ziemlich nervice integration-hell auf uns zukommen, wenn man dann die verschiedenen systeme LLM + RAG zusammenbringt, weils viele kleine Parameter gibt an denen man tweaken kann und der output aufgrund der LLMs nur bedingt interpretierbar ist (e.g. ne kleine änderung im prompt oder kontext kann unerwartet gute oder schlechte nachfolgen haben). fun fun fun